### Load required libraries and define simulations settings path

In [1]:
import numpy as np
import scipy.sparse as sparse
import pickle

sim_folder = 'dos/mpcM2'

### MPC controller settings

In [ ]:
from pathlib import Path

# Plant dynamic model matrices
A = np.zeros((42,42))
B = np.identity(42)
nx,nu = A.shape[0], B.shape[1]

# MPC settings
# Prediction horizon
npred = 4
# Weighting matrices
Q = (0.2/npred)*sparse.block_diag([1]*nx)
R = (1/npred)*sparse.block_diag([1]*nu)
# Controller output constraints
dumin, dumax = -0.25*np.ones(nu), 0.25*np.ones(nu)
umin, umax = -2*np.ones(nu), 2*np.ones(nu)

# MPC dictionary
data = {'MPC':{'A':A,'B':B,'Q':Q,'R':R,
              'npred':npred,
               'dumin':dumin,'dumax':dumax,
               'umin':umin,'umax':umax}}
# Pickles MPC data into string representation
with open(Path(sim_folder)/'MPC.pickle','wb') as f:
    pickle.dump(data,f)

### Load DOS and define define folder with simulation settings

In [8]:
import dos
sim = dos.DOS(sim_folder)

INFO:DOS:Reading config from dos/mpcM2/dos.yaml
INFO:DOS:New driver: M2
INFO:M2:New input: Txyz
INFO:M2:New input: Rxyz
INFO:DOS:New driver: wfs48
INFO:wfs48:New output: M12_RBM
INFO:wfs48:Output logged in!
INFO:DOS:New driver: MPC
INFO:MPC:New input: wfs48 data
INFO:MPC:New output: M2 Txyz
INFO:MPC:Output logged in!
INFO:MPC:New output: M2 Rxyz
INFO:MPC:Output logged in!
INFO:DOS:New driver: science
INFO:science:New output: wfe_rms
INFO:science:Output logged in!
INFO:science:New output: tiptilt
INFO:science:Output logged in!
INFO:science:New output: segment_tiptilt
INFO:science:Output logged in!
INFO:wfs48 data:Linked to M12_RBM from wfs48
INFO:M2 Txyz:Linked to Txyz from M2
INFO:M2 Rxyz:Linked to Rxyz from M2
INFO:DOS:Simulation setup for a duration of 8s @ 1Hz (8 steps)!


SIMCEO server received: Acknowledging connection from SIMCEO client!


In [9]:
# Start simulation
sim.start()

INFO:DOS:Pushing configuration to server
INFO:M2:GMT
INFO:wfs48:OP0
INFO:science:OP1
INFO:DOS:Initializing
INFO:M2:None
INFO:wfs48:None
INFO:science:None
INFO:DOS:Running
INFO:DOS:Terminating
INFO:M2:GMT deleted!
INFO:wfs48:OpticalPath deleted!
INFO:science:OpticalPath deleted!


Elapsed time: 3.840259075164795


In [10]:
M2_RBM = sim.logs.entries['wfs48']['M12_RBM'].timeSeries
M2_Txyz, M2_Rxyz = M2_RBM[1][:,:21], M2_RBM[1][:,-21:]
scale = 1.0
print('Txyz:')
for k in range(7):
    print('Seg#',k+1,':',scale*M2_Txyz[-1,0+(k*3):3+(k*3)])
    
print('\nRxyz:')
for k in range(7):
    print('Seg#',k+1,':',scale*M2_Rxyz[-1,0+(k*3):3+(k*3)])

Txyz:
Seg# 1 : [-9.89553696e-07  1.00000152e-06  9.73331223e-14]
Seg# 2 : [-6.53561353e-19  4.70291337e-23 -3.28583943e-23]
Seg# 3 : [-7.91965719e-24 -1.52832328e-23 -2.63070554e-24]
Seg# 4 : [ 1.01667614e-20 -9.54531527e-22 -1.20485734e-22]
Seg# 5 : [-7.88165333e-26  2.78013450e-24 -8.93265942e-26]
Seg# 6 : [ 2.17578800e-24  3.61558844e-27 -1.27085770e-27]
Seg# 7 : [ 1.83919001e-23 -5.66182780e-24  3.33671781e-26]

Rxyz:
Seg# 1 : [3.55714394e-13 2.32042157e-07 8.79642417e-07]
Seg# 2 : [-2.53695524e-23  1.53221020e-19  5.78349151e-19]
Seg# 3 : [ 6.84870499e-23 -5.00824315e-23 -3.29147679e-24]
Seg# 4 : [-2.29663201e-22 -2.36810876e-21 -9.31691685e-21]
Seg# 5 : [-1.12589881e-23 -3.38542956e-25  1.28855313e-28]
Seg# 6 : [-1.53570805e-26  9.31742025e-24 -1.06306259e-26]
Seg# 7 : [2.37176495e-23 7.70398532e-23 0.00000000e+00]


In [ ]:
print(M2_Txyz[-1],M2_Rxyz[-1])

### Plot simulation results
#### Mirror states

In [ ]:
# Plot mirror states
import matplotlib.pyplot as plt

M2_RBM = sim.logs.entries['wfs48']['M12_RBM'].timeSeries
M2_Txyz = M2_RBM[1][:,:21]
M2_Rxyz = M2_RBM[1][:,-21:]

plt.figure(figsize=(16,5))

plt.subplot(211)
plt.plot(M2_RBM[0],M2_Txyz,'.--')
plt.grid(True)
plt.ylabel('M2 Txyz state')
    
plt.subplot(212)
plt.plot(M2_RBM[0],M2_Rxyz,'.--')
plt.grid(True)
plt.ylabel('M2 Rxyz state')
    
plt.show()

In [ ]:
# Plot controller outputs
uTxyz = sim.logs.entries['MPC']['M2 Txyz'].timeSeries
uRxyz = sim.logs.entries['MPC']['M2 Rxyz'].timeSeries

fig = plt.figure(figsize=(16,5))

plt1, plt2 = fig.add_subplot(211), fig.add_subplot(212)

for k in range(7):
    for kk in range(3):
        #Txyz
        plt1.plot(uTxyz[1][k,kk,:],'.--')
        plt1.grid(True)
        plt1.set_ylabel('Txyz cmd')
        plt1.autoscale(enable=True, axis='y')
        #Rxyz
        plt2.plot(uRxyz[1][k,kk,:],'.--')
        plt2.grid(True)
        plt2.set_ylabel('Rxyz cmd')

plt.show()

In [ ]:
# Plot mirror states
import matplotlib.pyplot as plt

M2_RBM = sim.logs.entries['wfs48']['M12_RBM'].timeSeries
M2_Txyz = M2_RBM[1][:,:3]
M2_Rxyz = M2_RBM[1][:,21:24]

plt.figure(figsize=(16,5))

plt.subplot(211)
plt.plot(M2_RBM[0],M2_Txyz,'.--')
plt.grid(True)
plt.ylabel('M2 Txyz state')
    
plt.subplot(212)
plt.plot(M2_RBM[0],M2_Rxyz,'.--')
plt.grid(True)
plt.ylabel('M2 Rxyz state')
    
plt.show()